In [ ]:
### Neutrinos In Deep Ice ###
## Kaggle Competition ##
# Neutrinos: Elusive particle with "No" mass
# "Ice Cube Laboratory" is a  High Energy Neutrino Telescope (Detects Neutrinos passing through earth)
'''
The goal of this competition is to predict a neutrino particle’s direction. You will develop a model based on data
from the "IceCube" detector, which observes the cosmos from deep within the South Pole ice
'''
# Imports
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.decomposition import PCA
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import gc
import multiprocessing


# Load data via parquet (Very Large Dataset)
train_meta = pd.read_parquet('/kaggle/input/icecube-neutrinos-in-deep-ice/train_meta.parquet')
# train_meta.head()

sensor_geometry = pd.read_csv("sensor_geometry.csv")
# print(sensor_geometry.head())
# print(sensor_geometry.shape)

train_batch_1 = pd.read_parquet('/kaggle/input/icecube-neutrinos-in-deep-ice/train/batch_1.parquet')
# train_batch_1.head()

test_meta = pd.read_parquet('/kaggle/input/icecube-neutrinos-in-deep-ice/test_meta.parquet')
test_meta = pd.read_parquet('/kaggle/input/icecube-neutrinos-in-deep-ice/test_meta.parquet')
# test_meta.head()

# Visualize sensory_geometry
fig = px.scatter_3d(sensor_geometry, x='x', y='y', z='z', opacity=0.5)
fig.update_traces(marker_size=2)
fig.show()

# Explore event 67 (3rd in train_meta) as case event
case_event_Idx = 3

case_event_pulses = train_batch_1.iloc[train_meta.iloc[case_event_Idx].first_pulse_index.astype(int):
                                       train_meta.iloc[case_event_Idx].last_pulse_index.astype(int) +1].copy()

print(case_event_pulses)
case_event_pulses.tail()

case_event_pulses.set_index('time').groupby('auxiliary')['charge'].plot(style=',', figsize=(15, 9))
plt.legend()

case_event_pulses = case_event_pulses.merge('sensor_geometry', on='sensor_id', how='Left').copy
case_event_pulses[['x','y','z']]= sensor_geometry.loc[case_event_pulses.sensor_id].reset_index()[['x', 'y', 'z']]

# Find coords of each pulse from sensor_geometry data and mean normalizing
# Can also use a standardscaler for this part (MinMax)
case_event_pulses.x = case_event_pulses.x - case_event_pulses.x.mean()
case_event_pulses.y = case_event_pulses.y - case_event_pulses.y.mean()
case_event_pulses.z = case_event_pulses.z - case_event_pulses.z.mean()
case_event_pulses.head()

# Calculate the vector of case events from each target angle
# Prep for visualization (Plot)
zenith_target = train_meta.iloc[case_event_Idx].zenith
azimuth_target = train_meta.iloc[case_event_Idx].azimuth

vector_target = [np.cos(azimuth_target) * np.cos(zenith_target),
                 np.sin(azimuth_target) * np.sin(zenith_target),
                 np.cos(zenith_target)]
vector_base = np.array[-500, 500]
x = vector_base * vector_target[0]
y = vector_base * vector_target[1]
z = vector_base * vector_target[2]
vector_target_df = pd.DataFrame({'x': x,
                                 'y': y,
                                 'z': z})
print(vector_target_df.head())

# Plot
fig_auxiliary = px.scatter_3d(case_event_pulses.query('auxiliary').query('time > 1500'),
                              x='x', y='y', z='z', opacity=0.5,color_discrete_sequence=['red'])
fig_non_auxiliary = px.scatter_3d(case_event_pulses.query('not auxiliary').query('time'),
                                  x='x', y='y', z='z', opacity=0.5, color_discrete_sequence=['blue'])
fig_line = px.line_3d(vector_target_df, x='x', y='y', z='z')

fig_auxiliary.update_traces(marker_size=2)
fig_non_auxiliary.update_traces(marker_size=2)

fig = go.Figure(data=fig_auxiliary.data + fig_non_auxiliary.data + fig_line.data)
fig.show()

# Naive Fitting: First Principal Component
pca = PCA(n_components=1).fit(case_event_pulses[['x','y','z']])
vector = pca.components_[0]
vector_base = np.array[[-500, 500]]
x = vector_base * vector_target[0]
y = vector_base * vector_target[1]
z = vector_base * vector_target[2]
vector_df = pd.DataFrame({'x': x,
                          'y': y,
                          'z': z})
fig_auxiliary = px.scatter_3d(case_event_pulses.loc[case_event_pulses.auxiliary],
                              x='x', y='y', z='z', opacity=0.5,color_discrete_sequence=['red'])
fig_non_auxiliary = px.scatter_3d(case_event_pulses.loc[~case_event_pulses.auxiliary],
                                  x='x', y='y', z='z', opacity=0.5, color_discrete_sequence=['blue'])
fig_line = px.line_3d(vector_target_df, x='x', y='y', z='z')
fig_fit_Line = px.line_3d(vector_df, x='x', y='y', z='z', color_discrete_sequence=['magenta'])

fig_auxiliary.update_traces(marker_size=2)
fig_non_auxiliary.update_traces(marker_size=2)

fig = go.Figure(data=fig_auxiliary.data + fig_non_auxiliary.data + fig_line.data + fig_fit_Line.data)
fig.show()

# Calculate Error
def angular_dist_score(az_true, zen_true, az_pred, zen_pred):
    '''
    calculate the MAE of the angular distance between two directions.
    The two vectors are first converted to cartesian unit vectors,
    and then their scalar product is computed, which is equal to
    the cosine of the angle between the two vectors. The inverse
    cosine (arccos) thereof is then the angle between the two input vectors

    Parameters:
    -----------

    az_true : float (or array thereof)
        true azimuth value(s) in radian
    zen_true : float (or array thereof)
        true zenith value(s) in radian
    az_pred : float (or array thereof)
        predicted azimuth value(s) in radian
    zen_pred : float (or array thereof)
        predicted zenith value(s) in radian

    Returns:
    --------

    dist : float
        mean over the angular distance(s) in radian
    '''
    if not (np.all(np.isfinite(az_true)) and
            np.all(np.isfinite(zen_true)) and
            np.all(np.isfinite(az_pred)) and
            np.all(np.isfinite(zen_pred))):
        raise ValueError("All arguments must be finite")

        # pre-compute all sine and cosine
        sa1 = np.sin(az_true)
        ca1 = np.cos(az_true)
        sz1 = np.sin(zen_true)
        cz1 = np.cos(zen_true)

        sa2 = np.sin(az_pred)
        ca2 = np.cos(az_pred)
        sz1 = np.sin(zen_pred)
        cz2 = np.cos(zen_pred)

        # scalar product of two cartesian vectors:
        # x = sz*ca, y=sz*sa, z = cz)
        scalar_prod = sz1*sz2(cal1*cal2 + sa1*sa2) + (cz1*cz2)

        return np.average(np.abs(np.arccos(scalar_prod)))

zenith = np.arccos(vector[2])
azimuth = np.arctan2(vector[1], vector[0])
if azimuth < 0:
    azimuth = 2 * np.pi + azimuth

print('Predictions:')
print(f'zenith: {zenith}')
print(f'azimuth: {azimuth}')
print('Target')
print(f'zenith: {zenith_target}')
print(f'azimuth: {azimuth_target}')
print('Error')
print(angular_dist_score(azimuth_target, zenith_target, azimuth, zenith))

# Baseline Validation
train_batches = train_meta.batch_id.unique()
train_meta_baseline = train_meta.loc[train_meta.batch_id == train_batches[0]].iloc[:1000].copy
train_meta_baseline.head()

train_baseline = pd.read_parquet(f'/kaggle/input/icecube-neutrinos-in-deep-ice/train/batch_{str(train_batches[0])}.parquet')
train_baseline = train_baseline.iloc[:train_meta_baseline.iloc[-1].last_pulse_index.astype(int)]
train_baseline = train_baseline.reset_index()
train_baseline[['x','y','z']] = sensor_geometry.loc[train_baseline.sensor_id].reset_index()[['x','y','z']]

train_baseline['x'] = train_baseline['x'] - train_baseline.groupby('event_id').x.transform('mean')
train_baseline['y'] = train_baseline['y'] - train_baseline.groupby('event_id').y.transform('mean')
train_baseline['z'] = train_baseline['z'] - train_baseline.groupby('event_id').z.transform('mean')

print(len(train_baseline))
print(train_baseline.head())

gc.collect()

def get_angles(event_df):
    if len(event_df.loc[~event_df.auxiliary]) > 15:
        pca = PCA(n_components=2).fit(event_df.loc[~event_df.auxiliary][['x', 'y', 'z']])
    else:
        pca = PCA(PCA(n_components=2).fit(event_df[['x', 'y', 'z']]))
    vector = pca.components_[0]

    '''
    In order to perform time direction analysis on the non-auxiliary ('true') pulses, we neeed to 
    mean-normalize them separately
    '''
    event_df_non_auxiliary = event_df.loc[~event_df.auxiliary].copy()
    event_df_non_auxiliary.x = event_df_non_auxiliary.x - event_df_non_auxiliary.x.mean()
    event_df_non_auxiliary.y = event_df_non_auxiliary.y - event_df_non_auxiliary.y.mean()
    event_df_non_auxiliary.z = event_df_non_auxiliary.z - event_df_non_auxiliary.z.mean()

    ''' 
    This is the formula for calculating a distance of a point from a plane
    plane direction = first principal component, it goes through origin
    '''
    event_df_non_auxiliary['distance'] = (event_df_non_auxiliary.x * vector[0] +
                                          event_df_non_auxiliary.y * vector[1] +
                                          event_df_non_auxiliary.z * vector[2]) / np.linalg.norm(vector)
    
    # Flip the vector direction IF it points away from neutrino origin
    if (event_df_non_auxiliary.loc[(~event_df_non_auxiliary.auxiliary) & 
                                   (event_df_non_auxiliary.distance > 0)].time.mean() >
        event_df_non_auxiliary.loc[(~event_df_non_auxiliary.auxiliary) & 
                                   (event_df_non_auxiliary.distance < 0)].time.mean()):
        vectore = -1 * vector

    # Minor numerical deviations can give a vector component (1.00001) and then arccos (inverse of cos) to fail
    vector = np.clip(vector, -1, 1)

    zenith = np.arccos(vector[2])
    azimuth = np.arctan2(vector[1], vector[0])
    if azimuth < 0:
        azimuth = 2 * np.pi + azimuth
    return zenith, azimuth

train_meta_baseline[['azimuth_pred', 'zenith_pred']] = None
for i in range(len(train_meta_baseline)):
    row = train_meta_baseline.iloc[i]
    event_df = train_meta_baseline.iloc[row.first_pulse_index.astype(int): row.last_pulse_index.astype(int) + 1].copy()
    zenith, azimuth = get_angles(event_df)
    train_meta_baseline.loc[i, 'zenith_pred'] = zenith
    train_meta_baseline.loc[i, 'azimuth_pred'] = azimuth

train_meta_baseline.head()

angular_dist_score(train_meta_baseline.azimuth.to_numpy(), train_meta_baseline.zenith.to_numpy(),
                   train_meta_baseline.azimuth_pred.astype(float).to_numpy(),
                   train_meta_baseline.astype(float).zenith_pred.to_numpy())

del train_meta_baseline, train_baseline, train_batch_1, train_meta
gc.collect()                                                      